In [1]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from scipy.integrate import odeint
from scipy.optimize import fsolve
import sys

import numba

## FUNCTION CALL TREE

In [3]:
##REF -> Related to Reflection, UNDER cth_map.size != 0 condition (SAFE TO REMOVE, so far)
## -> Own comments
# -> OG commen
##REF_SEC -> In the section, dealing with reflection
##FUNC_CALL -> Function call (for user-defined functions)
##FUNC_SUPPLY -> Supplies function for odeint

## kerr_geodesic ->
    ##1 x_cam_to_x_BL -> # Transform camera cartesian coordinates to Boyer-Lindquist coordinates (# Get initial position in BL coordinates)
        ##1 x_cam_to_x_kerr -> # Rotate Cartesian camera coordinate system to cartesian coordinate system in which the z-axis is aligned with the black hole spin. The camera is located along the x-axis of the camera system.
            ##1 R_cam_to_kerr -> # Rotation matrix that rotates camera cartesian coordinates to Kerr cartesian coordinates
        ##2_FUNC_SUPPLY eqs_x_cam_to_x_BL -> # Equations for fsolve to transform cartesian to Boyer-Lindquist.

    ##2 inv_p_matrix -> # Inverse matrix for (##Initial) Boyer-Lindquist momentum variables, calcuated analytically

    ##3 initial_p_BL -> # Calculate the initial momentum from the camera pointing and photon energy
        ##1 R_cam_to_kerr -> # Rotation matrix that rotates camera cartesian coordinates to Kerr cartesian coordinates
    
    ##4_FUNC_SUPPLY kerr_dgl -> # Defines the differential equation for the Kerr geoesic. (## Returns the f (RHS) for odeint (from geodesic equation))
        ##1 kerr_functions -> # Defines expressions necessary for the Kerr differential equations
    
    ##5 BL_to_cam -> # Transfer Boyer-Lindquist to cartesian camera system
        ##1 R_kerr_to_cam -> # Rotation matrix that rotates Kerr cartesian coordinates to camera cartesian coordinates
    
    ##6_FUNC_SUPPLY kerr_dgl_reflect -> # Define reflected differential equation for Kerr:
        ##1 kerr_dgl ->
            ##1 kerr_functions
    
    ##7 BL_to_cam ->
        ##1 R_kerr_to_cam 
    
    ##8 I_cth -> # Get intensity from point closests to xyz on cth map sphere

    ##9 reflection ->   # Reflection coefficient with arguments:
                        # - multipole moment: l=0,1,2
                        # - reflection coefficient R0 between 0 and 1
                        # - polar angle theta.
    
    ##10 sphere_plot -> # Surface plot for 3D plot of horizon


## 15 functions here ->
    ## REF - 3 -> reflection, kerr_dgl_reflect, I_cth -> PROBABLY DON'T REQUIRE THESE
    ## NOT REF - 12 -> 🆙

## PARAMETERS

In [2]:
##REF -> r_reflect, r_cth, cth_map -> r_reflect is required, others are not
## l                        -> main.py -> Multipole moment set -> Use '0'                                                                                                  - ✅
## R0                       -> main.py -> <= 0.03 (for l = 0,1, for all 'a' considered in the paper - spans ~0 to ~1) (Eq. 5.3 - EHT Paper)                                - ✅
## M                        -> kerr_reflected_intensities.py -> Use '1' (as it estimates M from the image, which we cannot do here)                                        - ✅
## a                        -> main.py -> Rescaled in kerr_reflected_intensities.py -> Rescaling is perhaps not necessary -> Use '0.5' (for now - it should be modifiable) - ✅
## r_reflect -- REF         -> kerr_reflected_intensities.py -> '1.2*r_horizon'                                                                                            - ✅
## r_obs                    -> kerr_reflected_intensities.py -> '100.'                                                                                                     - ✅
## Theta_obs                -> main.py -> Use 'Theta_obs = 17./360.*2.*np.pi # M87 value.' (for now - it should be modifiable)                                             - ✅
## ang_size                 -> kerr_reflected_intensities.py -> 30.                                                                                                        - ✅
## E                        -> kerr_reflected_intensities.py - '1.'                                                                                                        - ✅
## h                        -> kerr_reflected_intensities.py -> Requires pix_size -> 'hv_set'                                                                              - ✅
## v                        -> kerr_reflected_intensities.py -> Requires pix_size -> 'hv_set'                                                                              - ✅
## r_cth -- REF             -> kerr_reflected_intensities.py -> 'r_BL_photonring'                                                                                          - ✅
## d_closeenough            -> kerr_reflected_intensities.py -> '1.5*pix_size'                                                                                             - ✅
## cth_map -- REF           -> Generated from image -> CANNOT USE THIS                                                                                                     - ❌
## mode                     -> kerr_geodesic() itself                                                                                                                      - ✅
## pix_size                 -> main.py -> For h and v -> 1.                                                                                                                - ✅

## TESTING THE CODE

In [38]:
## PARAMETERS
## -----------------------------------------------------------
## # Pixel size in muas:
pix_size = 1.

l = 0.
R0 = 0.03
M = 1.
a = 0.5

r_horizon = M + np.sqrt(M**2 - a**2)
r_reflect = 1.2*r_horizon

r_obs = 100.
Theta_obs = 17./360.*2.*np.pi
ang_size = 30.
E = 1.

## # Define radius where the cth map is created:
## r_photonring, r_BL_photonring = kerr_photon_ring_radius(M, a)
## r_cth = r_BL_photonring                                 ## NOT USED IN FIRST CALL IN kerr_intensities.py

## d_closeenough = 1.5*pix_size                            ## NOT USED IN FIRST CALL IN kerr_intensities.py
## cth_map = np.array([])                                  ## NOT USED IN FIRST CALL IN kerr_intensities.py
mode = 'geo_plot'                                          ## NOT USED IN FIRST CALL IN kerr_intensities.py -> Plotting outside kerr_geodesic()
## -----------------------------------------------------------

## # Number of pixels:
n_pix = int((2.*ang_size/pix_size)**2)

## # Sample list of h and v:
n_sample = int(np.sqrt(n_pix))
hv_set = [float(i) / (n_sample - 1) for i in range(n_sample)]

## # Scan over hv set to create the cth map: (NO, to create the geodesic, here)

@prof
# @numba.jit(nopython=True, parallel=True)
def test_eht():
    x_geo, y_geo, z_geo = [], [], []
    for h in hv_set:
        for v in hv_set:
            # Find cartesian coordinates along geodesic:
            x_geo, y_geo, z_geo, I = kerr_geodesic(l=l, R0=R0, M=M, a=a, r_reflect=r_reflect, r_obs=r_obs, Theta_obs=Theta_obs, ang_size=ang_size, E=E, h=h, v=v)
        
    return x_geo, y_geo, z_geo

## ---------- PROFILING STARTS HERE ----------
x, y, z = test_eht()
## ----------- PROFILING ENDS HERE -----------

## OUTPUT - PLOT & Some DATA
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
x_refl, y_refl, z_refl = sphere_plot(r_reflect) ##FUNC_CALL ✅ 10
ax.set_xlim([-r_obs, 1.05*r_obs])
ax.set_ylim([-r_obs, r_obs])
ax.set_zlim([-r_obs, r_obs])
ax.plot_surface(x_refl, y_refl, z_refl, color='black', alpha=0.2, label="r=r_h + epsilon")
ax.scatter(x, y, z, s=10, color='blue', label="Photon Geodesic")
ax.scatter(x[0], y[0], z[0], s=200, marker="o", color="black")
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
plt.legend()
plt.show()


## # Print a few important length scales
print("\nBlack hole spin: ", a/M)
print("Black hole mass: ", M)
print("Horizon: ", r_horizon)
print("Reflection surface: ", r_reflect)
## print("Close to horizon map: ", r_cth) ## NOT EVEN USED UP THERE, BUT CALCULATED
print("Photon ring Schwarzschild (3M): ", 3.*M)
## print("Photon ring Kerr: ", r_photonring) ## NOT EVEN USED UP THERE, BUT CALCULATED (COMMENTED OUT)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1mUntyped global name 'fsolve':[0m [1m[1mcannot determine Numba type of <class 'function'>[0m
[1m
File "<ipython-input-34-b1fcee149a9c>", line 6:[0m
[1mdef x_cam_to_x_BL(a, r0_cam, Theta_obs, ang_size, h, v):
    <source elided>
    v_guess = [r0_cam, Theta_obs, 0.]
[1m    BL = fsolve(eqs_x_cam_to_x_BL , v_guess, \
[0m    [1m^[0m[0m
[0m

In [21]:
## FROM utilities.py
# Radius of the photon ring for given mass and spin according to 0812.1328.
# THIS IS ONLY VALID WHEN BH SHADE IS APPROXIMATELY CIRCULAR, I.E. OBS ANGLE
# CLOSE TO ZERO AND/OR SPIN CLOSE TO ZERO.
def kerr_photon_ring_radius(M, J):
    r0_guess = 3.*M
    r0 = fsolve(xi_zero, r0_guess, args=(M, J))[0]
    eta0 = (4.*J**2*M*r0**3-r0**4*(r0-3.*M)**2)/J**2/(r0-M)**2 # Eq. 9.
    r_ring = np.sqrt(eta0 + J**2)
    return r_ring, r0

In [22]:
# Get Kerr geodesic.
## @numba.jit(nopython=True, parallel=True) ## Doesn't work, because 'odeint' is unrecognized
def kerr_geodesic(l, R0, M, a, r_reflect, r_obs, Theta_obs, ang_size, E, h, v, \
        r_cth=0., d_closeenough=0., cth_map=np.array([]), mode="no_plot"):

    # Outer horizon:
    r_horizon = M + np.sqrt(M**2 - a**2)

    # Radius at which we consider photon to be far enough from black hole to
    # use flat space approximation, i.e. no more light bending.
    r_flat = 100.*M

    # ODE solver parameters:
    abserr = 1.0e-8
    relerr = 1.0e-6
    s_stop = 2000.0
    numpoints = 3000

    # Affine parameter sampling:
    s = [s_stop * float(i) / (numpoints - 1) for i in range(numpoints)]

    # Get initial position in BL coordinates:
    r0, Theta0, phi0 = x_cam_to_x_BL(a, r_obs, Theta_obs, ang_size, h, v) ##FUNC_CALL ✅ 1

    # Inverse Matrix for intitial momentum in BL coordinates:
    Minv = inv_p_matrix(a, r0, Theta0, phi0) ##FUNC_CALL ✅ 2

    # Get initial momentum from E, h and v in flat space approximation for
    # camera location:
    p_r0, p_Theta0, p_phi0 = initial_p_BL(Theta_obs, E, Minv) ##FUNC_CALL ✅ 3

    # Pack up the parameters and initial conditions:
    m = [M, a, E]
    x0 = [r0, Theta0, phi0, p_r0, p_Theta0, p_phi0]
    x0_reflect = x0 ##REF

    # Unreflected rays get dummy default intensity to filter for them later.
    I = -1.

    # Get solutions from camera to reflection surface:
    reflected = False ##REF
    kerr_sol = odeint(kerr_dgl, x0, s, args=(m,), atol=abserr, \
            rtol=relerr) ##FUNC_SUPPLY ✅ 4
    x = []
    y = []
    z = []
    for si, xi in zip(s, kerr_sol):
        x_cam = BL_to_cam(a, Theta_obs, xi) ##FUNC_CALL ✅ 5
        if xi[0] < r_flat:
            if xi[0] > r_reflect:
                x.append(x_cam[0])
                y.append(x_cam[1])
                z.append(x_cam[2])
                if si > 0.95*s_stop:
                    print("Running out of parameter sampling. Abort!")
                    sys.exit(0)
            ## REF ⬇
            else:
                # Only explore reflection if cth_map is provided. Otherwise
                # intensity can't be determined.
                if cth_map.size != 0:

                    # BL coordinates where the light ray got reflected.
                    # Important to determine how intensity gets modified by
                    # reflection coefficient.
                    x0_reflect = xi
                    reflected = True

                break
    
    ## REF ⬇
    
    # Calculate reflected geodesic and use cth map:
    ##Not exploring relfection is already handled through cth_map.size != 0 condition above and below
    ##VARS : cth_map, reflected, x0_reflect
    ##FUNCS : kerr_dgl_reflect, I_cth, reflection
    if cth_map.size != 0 and reflected:

        # Get solutions from the reflection surface to twice r_cth in case there
        # was a reflection:
        kerr_sol_reflect = odeint(kerr_dgl_reflect, \
                x0_reflect, s, args=(m,), atol=abserr, rtol=relerr) ##FUNC_SUPPLY ##REF_SEC ✅ 6
        for si, xi in zip(s, kerr_sol_reflect):
            if xi[0] < 2.*r_cth and xi[0] > r_horizon:
                x_cam = BL_to_cam(a, Theta_obs, xi) ##FUNC_CALL ##REF_SEC ✅ 7
                x.append(x_cam[0])
                y.append(x_cam[1])
                z.append(x_cam[2])
            else:
                break

        # Reverse order list of coordinates produced by dgl solver, IF cth map
        # should be used BEFORE reflection:
        x = list(reversed(x))
        y = list(reversed(y))
        z = list(reversed(z))

        # Get coordinates of geodesic that are at same distance as the cth map:
        for i in range(len(x)):
            d = np.sqrt(x[i]**2+y[i]**2+z[i]**2)
            if d < r_cth:

                # Get intensity via cth map and reflection:
                I = I_cth(cth_map, np.array([x[i], y[i], z[i]]), d_closeenough) ##FUNC_CALL ##REF_SEC ✅ 8

                # Plot geodesic if it will contribute to the total flux to
                # check where the light rays come from:
                """
                if I > 0.:
                #if I > 0. and abs(h-0.5) < 0.01 and abs(v-0.5) < 0.01:
                    print((2*h-1)*ang_size, (2*v-1)*ang_size)
                    fig = plt.figure()
                    ax = fig.add_subplot(111, projection='3d')
                    x_refl, y_refl, z_refl = sphere_plot(r_reflect)
                    x_cth, y_cth, z_cth = sphere_plot(r_cth)
                    r_3Dplot = 1.5*r_cth
                    ax.set_xlim([-r_3Dplot, r_3Dplot])
                    ax.set_ylim([-r_3Dplot, r_3Dplot])
                    ax.set_zlim([-r_3Dplot, r_3Dplot])
                    ax.plot_surface(x_refl, y_refl, z_refl, color='black', \
                            alpha=0.3)
                    ax.plot_surface(x_cth, y_cth, z_cth, color='black', \
                            alpha=0.1)
                    ax.scatter(x, y, z, s=10, color='blue')
                    ax.set_xlabel('x')
                    ax.set_ylabel('y')
                    ax.set_zlabel('z')
                    plt.show()
                """

                # Modify intensity by reflection coefficient at reflection
                # surface:
                I = I*reflection(l, R0, x0_reflect[1]) ##FUNC_CALL ##REF_SEC ✅ 9

                break

    ## REF ⬆

    # Plot geodesic:
    if mode == "geo_plot":
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        x_refl, y_refl, z_refl = sphere_plot(r_reflect) ##FUNC_CALL ✅ 10
        ax.set_xlim([-r_obs, 1.05*r_obs])
        ax.set_ylim([-r_obs, r_obs])
        ax.set_zlim([-r_obs, r_obs])
        ax.plot_surface(x_refl, y_refl, z_refl, color='black', alpha=0.2, \
                label="r=r_h + epsilon")
        ax.scatter(x, y, z, s=10, color='blue', label="Photon Geodesic")
        ax.scatter(x[0], y[0], z[0], s=200, marker="o", color="black")
        ax.set_xlabel('x')
        ax.set_ylabel('y')
        ax.set_zlabel('z')
        #plt.legend()
        plt.show()
        ## plt.savefig("figures/kerr_geodesic.pdf", \
        ##         bbox_inches='tight')
        ## plt.close()
    
    return x, y, z, I

In [23]:
# from utilities import sphere_plot
# Surface plot for 3D plot of horizon:
# @numba.jit(nopython=True, parallel=True) ## Doesn't work
def sphere_plot(r):
    u = np.linspace(0, 2 * np.pi, 100)
    v = np.linspace(0, np.pi, 100)
    x = r * np.outer(np.cos(u), np.sin(v))
    y = r * np.outer(np.sin(u), np.sin(v))
    z = r * np.outer(np.ones(np.size(u)), np.cos(v))
    return x, y, z

In [24]:
# from kerr_dgl import kerr_functions
# Define expressions necessary for the Kerr differential equations:
@numba.jit(nopython=True, parallel=True)
def kerr_functions(M, a, r, Theta):
    T = Theta
    rs = 2*M
    rho2 = r**2 + a**2*np.cos(T)**2
    Delta = r**2 - rs*r + a**2

    # Functions that appear in the Kerr metric and Kerr geodesic equations:
    A = -1.*(1 - rs*r/rho2)
    B = a*r*rs*np.sin(T)**2/rho2
    C = rho2/Delta
    D = rho2
    F = np.sin(T)**2*(r**2+a**2+a*B)
    G = F - B**2/A
    H = B/A

    # Derivates of those functions:
    dDdr = 2*r
    dDdT = -a**2*np.sin(2*T)
    dAdr = -1.*(-rs*(1/D - r/D**2*dDdr))
    dAdT = -1.*(rs*r/D**2*dDdT)
    dBdr = a*rs*np.sin(T)**2*(1/D - r/D**2*dDdr)
    dBdT = a*rs*r*(np.sin(2*T)/D - np.sin(T)**2/D**2*dDdT)
    dCdr = 1/Delta*dDdr - D/Delta**2*(2*r-rs)
    dCdT = 1/Delta*dDdT
    dFdr = np.sin(T)**2*(2*r + a*dBdr)
    dFdT = np.sin(2*T)*(r**2+a**2+a*B) + np.sin(T)**2*a*dBdT
    dGdr = dFdr - 2*B/A*dBdr + B**2/A**2*dAdr
    dGdT = dFdT - 2*B/A*dBdT + B**2/A**2*dAdT
    dHdr = 1/A*dBdr - B/A**2*dAdr
    dHdT = 1/A*dBdT - B/A**2*dAdT

    # k and derivatives are composed of the functions A, B, C, D, F, G, H:
    k = [A, B, C, D, F, G, H]
    dkdr = [dAdr, dBdr, dCdr, dDdr, dFdr, dGdr, dHdr]
    dkdT = [dAdT, dBdT, dCdT, dDdT, dFdT, dGdT, dHdT]

    return k, dkdr, dkdT

In [25]:
##REF

# from reflection_coefficient import reflection
# Reflection coefficient with arguments:
# - multipole moment: l=0,1,2
# - reflection coefficient R0 between 0 and 1
# - polar angle theta.
def reflection(l, R0, Theta):

    # Monopole:
    if l == 0:
        R = R0

    # Dipole:
    elif l == 1:
        R = R0*abs(np.cos(Theta))

    # Quadrupole:
    elif l == 2:
        R = R0*abs(np.sin(Theta)*np.cos(Theta))

    else:
        print("Can only handle multipoles up to l=2 right now. Abort!")
        sys.exit(0)
    if R < 0. or R > 1.:
        print("Invalid reflection coefficient. Abort!")
        sys.exit(0)
    return R

In [26]:
# Defines the differential equation for the Kerr geoesic. s is the affine
# parameter of the geodesic. 
@numba.jit(nopython=True, parallel=True)
def kerr_dgl(x, s, m):
    
    # Coordinates:
    r, Theta, phi, p_r, p_Theta, p_phi  = x

    # Parameters:
    M, a, E = m

    # Get Kerr functions:
    k, dkdr, dkdT = kerr_functions(M, a, r, Theta) ##FUNC_CALL ✅
    A, B, C, D, F, G, H = k
    dAdr, dBdr, dCdr, dDdr, dFdr, dGdr, dHdr = dkdr
    dAdT, dBdT, dCdT, dDdT, dFdT, dGdT, dHdT = dkdT

    # Define dt/dlambda which is a function of the Kerr functions:
    tdot = E/A + H*p_phi

    # Create f = (r', Theta', phi', p_r', p_Theta', p_phi'):
    f = [ p_r, \
            p_Theta, \
            p_phi, \
            1/2./C*(dAdr*tdot**2 - 2*dBdr*tdot*p_phi - dCdr*p_r**2 - \
            2*dCdT*p_Theta*p_r + dDdr*p_Theta**2 + dFdr*p_phi**2), \
            1/2./D*(dAdT*tdot**2 - 2*dBdT*tdot*p_phi + dCdT*p_r**2 - \
            2*dDdr*p_Theta*p_r - dDdT*p_Theta**2 + dFdT*p_phi**2), \
            1/G*(E*dHdr*p_r + E*dHdT*p_Theta - dGdr*p_r*p_phi - \
            dGdT*p_Theta*p_phi)]

    return f

In [27]:
##REF

# Define reflected differential equation for Kerr:
def kerr_dgl_reflect(x, s, m):
    f = kerr_dgl(x, s, m) ##FUNC_CALL ##REF_SEC ✅
    return [f[0], f[1], f[2], -f[3], f[4], f[5]]

In [28]:
# Calculate the initial momentum from the camera pointing and photon energy.
# This is done in the flat space approximation.
@numba.jit(nopython=True, parallel=True)
def initial_p_BL(Theta_obs, E, Minv):

    # Initial momentum in camera coordinate system. Due to extreme hierachy
    # between distance to M87 (Mpc) and size of BH (AU) the light rays are
    # parallel to the x-axis for all practical purposes.
    p0_cam = [-E, 0., 0.]

    # Rotate to Kerr cartesian momentum:
    R = R_cam_to_kerr(Theta_obs) ##FUNC_CALL ✅
    p0_kerr = R.dot(p0_cam)

    # Transform Kerr cartesian momentum to Boyer-Lindquist momentum:
    p_BL = Minv.dot(p0_kerr)

    return p_BL


In [29]:
# Inverse matrix for Boyer-Lindquist momentum variables, calcuated analytically
# in ../mathematica/initial_momentum_matrix_inverse.nb.
@numba.jit(nopython=True, parallel=True) ## Doesn't work
def inv_p_matrix(a, r0, Theta0, phi0):
    denom = a**2 + 2.*r0**2 + a**2*np.cos(2.*Theta0)
    sqrtfac = np.sqrt(a**2 + r0**2)
    Minv = np.array([[2.*r0*sqrtfac*np.sin(Theta0)*np.cos(phi0)/denom, \
            2.*r0*sqrtfac*np.sin(Theta0)*np.sin(phi0)/denom, \
            2.*(a**2+r0**2)*np.cos(Theta0)/denom], \
            [2.*sqrtfac*np.cos(Theta0)*np.cos(phi0)/denom, \
            2.*sqrtfac*np.cos(Theta0)*np.sin(phi0)/denom, \
            -2.*r0*np.sin(Theta0)/denom], \
            [-np.sin(phi0)/np.sin(Theta0)/sqrtfac, \
            np.cos(phi0)/np.sin(Theta0)/sqrtfac, \
            0.]])
    return Minv


In [30]:
# Transfer Boyer-Lindquist to cartesian camera system: 
@numba.jit(nopython=True, parallel=True)
def BL_to_cam(a, Theta_obs, x_BL):
    # Transfer from Boyer-Lindquist to cartesian Kerr:
    x_kerr = [np.sqrt(x_BL[0]**2+a**2)*np.cos(x_BL[2])*np.sin(x_BL[1]), \
            np.sqrt(x_BL[0]**2+a**2)*np.sin(x_BL[2])*np.sin(x_BL[1]), \
            x_BL[0]*np.cos(x_BL[1])]
    # Transfer cartesian Kerr to cartesian camera: 
    R_inv = R_kerr_to_cam(Theta_obs) ##FUNC_CALL ✅
    x_cam = R_inv.dot(np.array(x_kerr))
    return x_cam

In [31]:
# Rotate Cartesian camera coordinate system to cartesian coordinate system in
# which the z-axis is aligned with the black hole spin. The camera is located
# along the x-axis of the camera system.
@numba.jit(nopython=True, parallel=True) ## Doesn't work
def x_cam_to_x_kerr(r0_cam, Theta_obs, ang_size, h, v):
    x_cam = np.array([r0_cam, (2*h-1)*ang_size, (2*v-1)*ang_size])
    R = R_cam_to_kerr(Theta_obs) ##FUNC_CALL ✅
    x_kerr = R.dot(x_cam)
    return x_kerr

In [32]:
# Rotation matrix that rotates camera cartesian coordinates to Kerr cartesian
# coordinates:
@numba.jit(nopython=True, parallel=True)
def R_cam_to_kerr(Theta_obs):
    if Theta_obs > np.pi/2. or Theta_obs < 0.:
        print("Theta_obs is out of range [0,pi/2]. Abort!")
        sys.exit(0)
    R = np.array([[np.sin(Theta_obs), 0., -np.cos(Theta_obs)], \
            [0., 1., 0.], \
            [np.cos(Theta_obs), 0., np.sin(Theta_obs)]])
    return R

In [33]:
# Rotation matrix that rotates Kerr cartesian coordinates to camera cartesian
# coordinates:
@numba.jit(nopython=True, parallel=True)
def R_kerr_to_cam(Theta_obs):
    if Theta_obs > np.pi/2. or Theta_obs < 0.:
        print("Theta_obs is out of range [0,pi/2]. Abort!")
        sys.exit(0)
    R = np.array([[np.sin(Theta_obs), 0., np.cos(Theta_obs)], \
            [0., 1., 0.], \
            [-np.cos(Theta_obs), 0., np.sin(Theta_obs)]])
    return R

In [34]:
# Transform camera cartesian coordinates to Boyer-Lindquist coordinates:
@numba.jit(nopython=True, parallel=True)
def x_cam_to_x_BL(a, r0_cam, Theta_obs, ang_size, h, v):
    x_kerr = x_cam_to_x_kerr(r0_cam, Theta_obs, ang_size, h, v) ##FUNC_CALL ✅
    v_guess = [r0_cam, Theta_obs, 0.]
    BL = fsolve(eqs_x_cam_to_x_BL , v_guess, \
            args=(a, x_kerr[0], x_kerr[1], x_kerr[2])) ##FUNC_SUPPLY ✅
    r0, Theta0, phi0 = BL[0], BL[1], BL[2]
    if r0 < 0.:
        print("Negative Boyer-Lindquist radial coordinate. Abort!")
        sys,exit(0)
    return r0, Theta0, phi0

In [35]:
# Equations for fsolve to transform cartesian to Boyer-Lindquist. z is the
# parameter vector build by [a, x, y, z].
# @numba.jit(nopython=True, parallel=True) ## Doesn't work, because 'fsolve' is not recognized
def eqs_x_cam_to_x_BL(v, *z):
    r0, Theta0, phi0 = v
    return (np.sqrt(r0**2+z[0]**2)*np.sin(Theta0)*np.cos(phi0) - z[1], \
            np.sqrt(r0**2+z[0]**2)*np.sin(Theta0)*np.sin(phi0) - z[2], \
            r0*np.cos(Theta0) - z[3])

In [36]:
##REF

# Get intensity from point closests to xyz on cth map sphere.
def I_cth(cth_map, xyz, d_closeenough):
    xyz = np.asarray(xyz)
    cth_xyzs = cth_map[:,:-1]
    deltas = cth_xyzs - xyz
    dists_to_point = np.einsum('ij,ij->i', deltas, deltas)
    idx = dists_to_point.argmin()
    # Only return intensity of nearest point of cth map if xyz is not too far
    # away:
    if np.sqrt(dists_to_point[idx]) < d_closeenough:
        I = cth_map[idx][-1]
    else:
        I = 0.
    return I

In [37]:
import cProfile, pstats, io
from functools import wraps

def prof(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        datafn = func.__name__ + ".prof"
        path = 'profdata/' + datafn
        pr = cProfile.Profile()
        ret_val = pr.runcall(func, *args, **kwargs)
        s = io.StringIO()
        ps = pstats.Stats(pr, stream=s).sort_stats('cumulative')
        ps.print_stats()

        with open(path, 'w') as report:
            report.write(s.getvalue())

        return ret_val

    return wrapper